# Set Up

In [ ]:
# !pip install pyspark

In [ ]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Import functions from pyspark.sql.functions
from pyspark.sql.functions import *

# For using window functions for aggregations or rankings
from pyspark.sql.window import Window

# For using data types when defining schemas or manipulating columns
from pyspark.sql.types import *

# Create a SparkSession, which is the entry point to programming Spark with the Dataset and DataFrame API
spark = SparkSession.builder \
    .appName("Data Cleaning with PySpark") \
    .getOrCreate()

In [ ]:
# Load dataset
df = spark.read.csv('kickstarter_data.csv', header=True, inferSchema=True)

# First 5 rows of the dataframe
df.show(5)

+---+----------+--------------------+--------------+-------------+--------+----------+-------+-------------------+-------+--------+-------+-------+-----------+----------------+-------------+
|_c0|        ID|                name|      category|main_category|currency|  deadline|   goal|           launched|pledged|   state|backers|country|usd pledged|usd_pledged_real|usd_goal_real|
+---+----------+--------------------+--------------+-------------+--------+----------+-------+-------------------+-------+--------+-------+-------+-----------+----------------+-------------+
|  0|1000002330|The Songs of Adel...|        Poetry|   Publishing|     GBP|2015-10-09| 1000.0|2015-08-11 12:12:28|    0.0|  failed|      0|     GB|        0.0|             0.0|      1533.95|
|  1|1000003930|Greeting From Ear...|Narrative Film| Film & Video|     USD|2017-11-01|30000.0|2017-09-02 04:43:57| 2421.0|  failed|     15|     US|      100.0|          2421.0|      30000.0|
|  2|1000004038|      Where is Hank?|Narrativ

**ANNOTATION**

Dataframe Columns:
    - `goal`: Goal set at the launched time.

    - `pledge`: Total amount of funding the project successfully called.

    - `backers`: Number of investors that fund the project.

    - `usd pledged`: conversion in US dollars of the pledged column (conversion done by kickstarter).

    - `usd_pledge_real`: conversion in US dollars of the pledged column (conversion from Fixer.io API).

    - `usd_goal_real`: conversion in US dollars of the goal column (conversion from Fixer.io API).

The dataset is acquired from Kaggle.com. You can visit it here: https://www.kaggle.com/kemical/kickstarter-projects

# A. OVERVIEW AND CLEAN

## **A.1** - Remove unwanted observations

### Question 1

We have many columns for the pledge and goal with different conversions.

For this analysis, we choose to keep only `usd_pledged_real` and `usd_goal_real`.

Write one line of code to drop the columns `goal`, `pledged`, `usd pledged`.

In [ ]:
# YOUR CODE HERE

### Question 2

For future convenience, let's rename the columns as follows:

- `usd_pledged_real` --> `pledged`
- `usd_goal_real` --> `goal`

Write your code to do that below.

In [ ]:
# YOUR CODE HERE

## **A.2** - Structural Error, Correct Datatype
---

### Question 3

Write one line of code to print the overall information of the dataset. Are there any columns that you feel like they have the wrong datatype?

In [ ]:
# YOUR CODE HERE

### Question 4

Convert the `launched` and `deadline` to `datetime` datatype.

In [ ]:
# YOUR CODE HERE

Check info one more time to make sure everything goes as plan.

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: string (nullable = true)
 |-- country: string (nullable = true)
 |-- usd pledged: string (nullable = true)
 |-- usd_pledged_real: string (nullable = true)
 |-- usd_goal_real: string (nullable = true)



## **A.3** - Handling Missing Values
---

### Question 5

Give the number of null values in *each* column.

In [ ]:
# YOUR CODE HERE

### Question 6

Write one line of code to fill all the `NaN` values in name with `Unknown`.

In [ ]:
# YOUR CODE HERE

## **A.4** - Handling errors, corrupted data
---

Scanning through each column to find abnormalities and fix them. Simple as that.

### Question 7

Let's start with `category`. Write an expression to display the frequency of the value in the column `category`. (The unique values and how many times they appear)

In [ ]:
# YOUR CODE HERE

### Question 8

Do the same to check abnormalities in the column `country`.

In [ ]:
# YOUR CODE HERE

### Question 9

Write an expression to select all rows with that weird value above (`N,0"`).

In [ ]:
# YOUR CODE HERE

### Question 10

Write one line of code to return the ***unique currencies*** of the projects that have country as `N,0"`?

In [ ]:
# YOUR CODE HERE

### Question 11

Our mission is apply a check function onto each row of the country-N0" part.

First, define a function that takes in a whole data row.

- If currency is `USD` ---> country is `US`
- If currency is `AUD` ---> country is `AU`
- If currency is `CAD` ---> country is `CA`
- If currency is `GBP` ---> country is `GB`
- If currency is `SEK` ---> country is `SE`
- If currency is `DKK` ---> country is `DK`
- If currency is `NZD` ---> country is `NZ`
- If currency is `NOK` ---> country is `NO`
- If currency is `CHF` ---> country is `CH`
- If currency is `EUR` ---> country is `DE`

In the `EUR` case, we choose to replace by the mode --- `DE` (Within projects that in `EUR`, the most are from `DE` -- Germany)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# YOUR CODE HERE

### Question 12

Save result using .write.parquet()